In [1]:
import keplerSettings as keplerSettings

In [ ]:
import findspark
findspark.init(keplerSettings.sparkHome)

In [2]:
from astropy.io import fits

In [3]:
#import astroprov
import sys
import datetime
import time
import collections
import subprocess

import numpy as np
import os

from astropy.table import Table
from astropy.table import Column
from astropy.io import ascii

import sqlite3
from pandas import DataFrame

import mysql.connector
from mysql.connector import Error
from mysql.connector import pooling

from pyspark.sql import Row
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark import SparkConf

In [4]:
defaultDirectory = keplerSettings.sextractorDirectory

In [5]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars '+ keplerSettings.mysqlSQLConnector+'  pyspark-shell'
config = SparkConf().setAll(keplerSettings.sparkConfig) # ("spark.sql.execution.arrow.enabled", "true")


In [6]:
sc = SparkContext(appName='App',conf=config)
sqlContext = SQLContext(sc)

In [7]:
mags = keplerSettings.magRangeSQLnaming
for mag in mags:
    dataframe_mysql = sqlContext.read.format("jdbc").option("url",  keplerSettings.databaseLoc).option("driver", "com.mysql.jdbc.Driver").option("dbtable", "starlist_{}".format(mag)).option("user", keplerSettings.databaseUsername).option("password", keplerSettings.databasePassword).load()
    dataframe_mysql.registerTempTable('starlist_{}'.format(mag))

In [8]:
def innerJoin(tableName,totGross,totGrossOriginal,variableList,mag,baseNum,diffSize=1,xlength=keplerSettings.astroImageXlength,ylength=keplerSettings.astroImageYlength,weight=0.5):

    nsegs = keplerSettings.nsegs
    nperseg = keplerSettings.totalObjects/(nsegs**2)
    actualtTotObjects = nperseg*nsegs*nsegs
    #mag = variableList[6]
    temptableName = tableName[0:5]+tableName[-1]
    OGtemptableName = 'original'
    OGdf = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE FROM {0} INNER JOIN starlist_{1} ON {0}.X_IMAGE BETWEEN starlist_{1}.X_POS_MIN AND starlist_{1}.X_POS_MAX AND {0}.Y_IMAGE BETWEEN starlist_{1}.Y_POS_MIN AND starlist_{1}.Y_POS_MAX".format(OGtemptableName,abs(mag)))
    OGdf.registerTempTable(OGtemptableName+'match')
    sqlContext.cacheTable(OGtemptableName+'match')
    
    OGtotNumDF = sqlContext.sql("SELECT * FROM {}".format(OGtemptableName))
    OGtotNumber = OGtotNumDF.count()
    
    df = sqlContext.sql("SELECT NUMBER, MAG_BEST, X_IMAGE, Y_IMAGE FROM {0} INNER JOIN starlist_{1} ON {0}.X_IMAGE BETWEEN starlist_{1}.X_POS_MIN AND starlist_{1}.X_POS_MAX AND {0}.Y_IMAGE BETWEEN starlist_{1}.Y_POS_MIN AND starlist_{1}.Y_POS_MAX".format(temptableName,abs(mag)))
    df.registerTempTable(temptableName+'match')
    sqlContext.cacheTable(temptableName+'match')
    totNumber = df.count()
    #print('tot',totNumber)
    
    xsegment = xlength/nsegs
    ysegment = ylength/nsegs
    
    accuracyScore = (float(OGtotNumber)/float(baseNum))
    #print('og',OGtotNumber,'base',baseNum,'tot',totNumber)
    completenessScore = 0.
    #totSeg = []
    border = keplerSettings.border
    #df.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='result_{}'.format(tableName),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    for xsegs in range(nsegs):
        xmin = int(xsegs*xsegment)+border
        xmax = int(xmin+xsegment)+border
        for ysegs in range(nsegs):
            ymin = int(ysegs*ysegment)+border
            ymax = int(ymin+ysegment)+border
            
            OGdf = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(OGtemptableName+'match',xmin,xmax,ymin,ymax))
            OGnumber = OGdf.count()
            df = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName+'match',xmin,xmax,ymin,ymax))
            number = df.count()
            simulated = number-OGnumber
            if simulated >  nperseg:
                simulated =  nperseg 
            completenessScore +=  simulated #abs(simulated-62)
            
            variableList.append(simulated)
            
#             dftot = sqlContext.sql("SELECT NUMBER FROM {0} WHERE {0}.X_IMAGE BETWEEN {1} AND {2} AND {0}.Y_IMAGE BETWEEN {3} AND {4}".format(temptableName,xmin,xmax,ymin,ymax))
#             number = df.count()
#             #variableList.append(number)
#             totSeg.append(number)

            # need to create some place to write to

    
    
#    variableList.extend(totSeg)
    #astroprov.provcall([tableName,'starlist_{}'.format(abs(mag))],['result_{}'.format(tableName)],"innerJoin_Python2Python_SQ_tmpl.provn","innerJoin",provDir)
    #compScore = 1. - (completenessScore/992.)
    compScore = 1. - (float(completenessScore)/float(actualtTotObjects))

    score = (weight*compScore) + ((1.-weight)*accuracyScore)
    #print(score,compScore,accuracyScore)
    sqlContext.uncacheTable(temptableName+'match')
    sqlContext.dropTempTable(temptableName+'match')
    sqlContext.uncacheTable(OGtemptableName+'match')
    sqlContext.dropTempTable(OGtemptableName+'match')
    sqlContext.uncacheTable(temptableName)
    sqlContext.dropTempTable(temptableName)
    sqlContext.uncacheTable(OGtemptableName)
    sqlContext.dropTempTable(OGtemptableName)
    variableList.append(totNumber)
    variableList.append(totGross)
    variableList.append(OGtotNumber)
    variableList.append(totGrossOriginal) 
    variableList.append(score)
    variableTuple = tuple(variableList)
    #print(variableTuple)
    return [variableTuple,score]


def makeConfig(valList,tableName='table',defaultDir=defaultDirectory,attributeList=keplerSettings.sextractorAttributeList):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    cname = 'CATALOG_NAME'
    catName = 'test1.cat'
    confName = 'default_1.sex'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    data = newData

    cname = 'PARAMETERS_NAME'
    nameLoc = data.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' autodefault.param ' + data[endLoc:]
    
    #print(newData)
    with open(defaultDir+confName,'w') as f:
        f.write(newData)
    return(confName,tableName,catName)

def findObjects(confName,tableName,catName,defaultDir=defaultDirectory,imagename='/home/mj1e16/iraf/editedImage5000.fits',original='no',base='no'):
    
    os.chdir(defaultDir)
    subprocess.call(['sex',imagename,'-c',confName])
    assoc = Table.read(catName,format='ascii.sextractor')
    df = assoc.to_pandas()
    df2 = df[df['MAG_BEST'] != 99.0000]
    df_spark = sqlContext.createDataFrame(df2)
    temptableName = tableName[0:5]+tableName[-1]
    if original == 'yes':
        temptableName = 'original'
    if base == 'yes':
        temptableName = 'baseSettings'
    df_spark.registerTempTable(temptableName)
    totGross = df_spark.count()
    sqlContext.cacheTable(temptableName)

    return totGross

def alltogethernow(valList,tableName,IMAGE,variableList,originalImage,mag,atList,baseNum,weight=0.5):
    Names = makeConfig(valList,tableName=tableName,attributeList=atList)
    totGrossOriginal = findObjects(Names[0],Names[1],Names[2],imagename=originalImage,original='yes')
    totGross = findObjects(Names[0],Names[1],Names[2],imagename=IMAGE) # confName tabName catname
    finalTuple = innerJoin(Names[1],totGross,totGrossOriginal,variableList,mag,baseNum,weight=weigth)
    return finalTuple # named final tuple but actually [finaltuple,score]

In [9]:
def hotpantsQuality(image):
    hdu = fits.open(image)
    imData = hdu[0].data
    imData = abs(imData)
    totVal = sum(imData)
    totVal = sum(totVal)
    return totVal

In [10]:
valList = keplerSettings.sextractorValueList

In [11]:
def evaluateImage(valList,simImage,OGImage,ccd,median,norm,minmag,smallName='newRun',atList=keplerSettings.sextractorAttributeList,dbtabname='finalresultsTable',weight=0.5):
    
    finalTableTuples = []
    t0 = time.time()

    for x0 in range(len(valList[0])):
        for x1 in range(len(valList[1])):
                name = smallName+'_'+str(x0)+'_'+str(x1)+'_'
                for x2 in range(len(valList[2])):
                    variableList = [float(valList[0][x0]),float(valList[1][x1]),valList[2][x2],ccd,median,minmag,norm]
                    fullname = name+str(x2)
                    fullValList = [valList[0][x0],valList[1][x1],valList[2][x2]]
                    print(fullname)
                    try:
                        alltogethernowResults = alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,variableList=variableList,originalImage=OGImage,mag=minmag,atList=atList,weight=weight)
                        finalTableTuples.append(alltogethernowResults[0])
                        score = alltogethernowResults[1]
                    except RuntimeError:
                        with open('redos','a') as f:
                            bigString = simImage+','+str(fullname)+','+str(fullValList)
                            f.write(bigString)
                        print('Redo with valList = ',fullValList)
                        pass

    print(finalTableTuples)
    rdd = sc.parallelize(finalTableTuples)
    kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
                                   minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[11]),
                                   xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
                                   xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),
                                   totNum=int(x[23]),totGross=int(x[24]),OGtot=int(x[25]),OGtotGross=int(x[26]),score=int(x[27])))

    schemaKepler = sqlContext.createDataFrame(kepler)
    #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
    schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResultsNewRun_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    
    print(time.time()-t0)


In [12]:
def oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName='newRun',starting=[0,0,0],atList=keplerSettings.sextractorAttributeList,weight=0.5):
    
    fullname = smallName+'_'+str(starting[0])+'_'+str(starting[1])+'_'+str(starting[2])               
    variableList = [float(valList[0][starting[0]]),float(valList[1][starting[1]]),valList[2][starting[2]],ccd,median,minmag,norm]
    fullValList = [valList[0][starting[0]],valList[1][starting[1]],valList[2][starting[2]]]
    print(fullname)
    print(starting)
    try:
        alltogethernowResults = alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,variableList=variableList,originalImage=OGImage,mag=minmag,atList=atList,baseNum=baseNum,weight=weight)
        finalTableTuples = alltogethernowResults[0]
        baseScore = alltogethernowResults[1]
    except RuntimeError:
        with open('/home/mj1e16/keplerPhotometry/redos.txt','a') as f:
            bigString = simImage+','+str(fullname)+','+str(fullValList)
            f.write(bigString)
            basescore = -1
        print('Redo with valList = ',fullValList)
        pass
    print(baseScore)
    return [finalTableTuples,baseScore]

In [13]:
# def evaluateImageHillClimbVersionOne(valList,simImage,OGImage,ccd,median,norm,minmag,smallName='newRun',atList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME'],starting=[0,0,0]):
    
#     finalTableTuples = []
#     t0 = time.time()
#     scores = []
    
#     for loop in range(2):
#         for x in range(len(valList)):
#             scorePerVal = []
#             for y in range(len(valList[x])):
#                 starting[x] = y
#                 fullname = smallName+'_'+str(starting[0])+'_'+str(starting[1])+'_'+str(starting[2])                
#                 variableList = [float(valList[0][starting[0]]),float(valList[1][starting[1]]),valList[2][starting[2]],ccd,median,minmag,norm]
#                 fullValList = [valList[0][starting[0]],valList[1][starting[1]],valList[2][starting[2]]]
#                 print(starting)
#                 print(fullname)
#                 try:
#                     alltogethernowResults = alltogethernow(fullValList,tableName=fullname,IMAGE=simImage,variableList=variableList,originalImage=OGImage,mag=minmag,atList=atList)
#                     finalTableTuples.append(alltogethernowResults[0])
#                     scorePerVal.append(alltogethernowResults[1])
#                 except RuntimeError:
#                     with open('redos','a') as f:
#                         bigString = simImage+','+str(fullname)+','+str(fullValList)
#                         f.write(bigString)
#                         scorePerVal.append(-1)
#                     print('Redo with valList = ',fullValList)
#                     pass
#             goodScores = [sco for sco in scorePerVal if x >= 0]
#             best = scorePerVal.index(min(goodScores))
#             starting[x] = best

#     print(finalTableTuples)
#     rdd = sc.parallelize(finalTableTuples)
#     kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
#                                    minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[12]),
#                                    xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
#                                    xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),totNum=int(x[23]),totGross=int(x[24]),OGtotGross=int(x[25])))

#     schemaKepler = sqlContext.createDataFrame(kepler)
#     #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
#     schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResultsNewRun_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    
#     print(time.time()-t0)
#     return starting


In [14]:
# def evaluateImageHillClimbVersionOne(valList,simImage,OGImage,ccd,median,norm,minmag,smallName='newRun',atList=['DETECT_THRESH','DETECT_MINAREA','FILTER_NAME'],starting=[0,0,0]):
    
#     finalTableTuples = []
#     t0 = time.time()
#     scores = []
    
#     startingBest = 10**10
#     best = 10**9
    
#     while startingBest > best:
#         startingBest = best
#         for x in range(len(valList)):
#             scorePerVal = []
#             for y in range(len(valList[x])):
#                 starting[x] = y
#                 baseEvaluate = oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,smallName=smallName,starting=starting,atList=atList)

#                 finalTableTuples.append(baseEvaluate[0])
#                 scorePerVal.append(baseEvaluate[1])

#             goodScores = [sco for sco in scorePerVal if x >= 0]
#             best = scorePerVal.index(min(goodScores))
#             bestList = [i for i, val in enumerate(goodScores) if val == min(goodScores)]
#             if len(bestList) == 1:
#                 best = bestList[0]
#             else:
#                 best = int(np.median(bestList))
#             starting[x] = best
#             print(scorePerVal)

#     print(finalTableTuples)
    
#     rdd = sc.parallelize(finalTableTuples)
#     kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
#                                    minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[11]),
#                                    xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
#                                    xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),
#                                    totNum=int(x[23]),totGross=int(x[24]),OGtot=int(x[25]),OGtotGross=int(x[26]),score=int(x[27])))

#     schemaKepler = sqlContext.createDataFrame(kepler)
#     #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
#     schemaKepler.write.format('jdbc').options(url='jdbc:mysql://localhost/Kepler',driver='com.mysql.jdbc.Driver',dbtable='finalResultsSimpleLoop_{}_{}_{}'.format(ccd,median,abs(minmag)),user='mj1e16',password='[sqlT1G3R]').mode('append').save()
    
#     print(time.time()-t0)
#     return starting


In [15]:
def evaluateImageHillClimbVersionTwo(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName='newRun',starting=[0,0,0],atList=keplerSettings.sextractorAttributeList,dbtabname='finalresultsTable',weight=0.5):
    
    finalTableTuples = []
    t0 = time.time()
    scores = []
    
    baseEvaluate = oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName=smallName,starting=starting,atList=atList,weight=weight)
    finalTableTuples.append(baseEvaluate[0])
    baseScore = baseEvaluate[1]
    startScore = baseScore + 1
    
    plusMinus = [-1,1]
    while baseScore < startScore:
        startScore = baseScore
        for x in range(len(valList)):
            for y in plusMinus:
                delta = -1
                while delta < 0:
                    minus = [z for z in starting]
                    minus[x] += y
                    if 0 < minus[x] < len(valList[x]):
                        minusEvaluate = oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName=smallName,starting=minus,atList=atList,weight=weight)
                        finalTableTuples.append(minusEvaluate[0])
                        minusScore = minusEvaluate[1]

                        delta = minusScore - baseScore
                        print('delta',delta,'starting',minus)
                        randomVals = [random.randint(0,len(valList[0])-1),random.randint(0,len(valList[1])-1),random.randint(0,len(valList[2])-1)]
                        randomEvaluate = oneSettingEvaluate(valList,simImage,OGImage,ccd,median,norm,minmag,baseNum,smallName=smallName,starting=randomVals,atList=atList,weight=weight)
                        finalTableTuples.append(randomEvaluate[0])
                        randomScore = randomEvaluate[1]

                        if randomScore < baseScore:
                            starting = randomVals
                            delta = randomScore - baseScore
                            baseScore = randomScore

                        elif minusScore < baseScore:
                            starting = minus
                            baseScore = minusScore
                    else:
                        break
                
                

    print(finalTableTuples)
    print(baseScore)
    print(starting)
    rdd = sc.parallelize(finalTableTuples)
    kepler = rdd.map(lambda x: Row(detectThresh=x[0],detectMinarea=x[1],filterName=x[2],ccd=x[3],median=x[4],medianQuality=x[5],
                                   minmag=x[6], xy_0=int(x[7]),xy_1=int(x[8]),xy_2=int(x[9]),xy_3=int(x[10]),xy_4=int(x[11]),
                                   xy_5=int(x[12]),xy_6=int(x[13]),xy_7=int(x[14]),xy_8=int(x[15]),xy_9=int(x[16]),xy_10=int(x[17]),
                                   xy_11=int(x[18]),xy_12=int(x[19]),xy_13=int(x[20]),xy_14=int(x[21]),xy_15=int(x[22]),
                                   totNum=int(x[23]),totGross=int(x[24]),OGtot=int(x[25]),OGtotGross=int(x[26]),score=int(x[27])))

    schemaKepler = sqlContext.createDataFrame(kepler)
    #print('finalResults_{}_{}_{}'.format(ccd,median,minmag))
    schemaKepler.write.format('jdbc').options(url=keplerSettings.databaseLoc,driver='com.mysql.jdbc.Driver',dbtable=dbtabname,user=keplerSettings.databaseUsername,password=keplerSettings.databasePassword).mode('append').save()
    
    print(time.time()-t0)
    return starting

In [19]:
ccd = keplerSettings.ccdExtensions
median = [1]
mags = keplerSettings.magRange

In [134]:
bestSettings = []
baseSettings = [valList[0][0],valList[1][0],valList[2][0]]

c = ccd[0]
med = median[0]
norm = 1.0
ogim = keplerSettings.simulatedImageDirectory+'diff_{}_{}.fits'.format(c,med)
#weight = keplerSettings.Weight
starting = [4,4,14]

# baseNames = makeConfig(baseSettings,tableName='baseTable')
# baseNum = findObjects(baseNames[0],baseNames[1],baseNames[2],imagename=ogim,base='yes')
#startRanges = [[0,0,0],[-1,-1,-1],] think about the start ranges
for starting in startRanges:
    for monte in range(10):
        bestsetWeight = []
        for w in range(10):
            weight = w/10.
            bestsettings = []
            for mag in mags:    
                baseNames = makeConfig(baseSettings,tableName='baseTable')
                baseNum = findObjects(baseNames[0],baseNames[1],baseNames[2],imagename=ogim,base='yes')
                sqlContext.cacheTable('starlist_{}'.format(abs(mag)))
                imName = keplerSettings.simulatedImageDirectory+'diff_{}_{}{}_alt.fits'.format(c,med,mag)
                tabName = keplerSettings.databaseTableName+'hillClimb_{}_{}_{}_{}'.format(c,med,abs(mag),weight)
                bestSettings.append(evaluateImageHillClimbVersionTwo(valList,imName,ogim,'ccd_{}'.format(c),'median_{}'.format(med),norm,mag,baseNum,smallName=tabName,starting=starting,weight=weight))
                sqlContext.uncacheTable('starlist_{}'.format(abs(mag)))
                starting = bestSettings[-1]
            bestsetWeight.append(bestSettings)
        with open('/home/mj1e16/keplerPhotometry/starting_{}_ccd_{}_hillClimbRes.py'.format(starting,c),'a') as f:
            for y in range(len(bestSettings)):
                for x in range(len(bestsetWeight))
                    f.write('\nmonte-{}-mag-{} = '.format(x,mags[y])+str(bestsetWeight[x][y]))
                f.write('\n')


newRun5_44_1_-7_4_6_12
[4, 6, 12]
('og', 301, 'base', 19902, 'tot', 897)
(0.06350963471007941, 0.11189516129032262, 0.015124108129836198)
0.0635096347101
newRun5_44_1_-7_3_6_12
[3, 6, 12]
('og', 422, 'base', 19902, 'tot', 899)
(0.06554146568184102, 0.1098790322580645, 0.021203899105617527)
0.0655414656818
('delta', 0.002031830971761614, 'starting', [3, 6, 12])
newRun5_44_1_-7_9_3_5
[9, 3, 5]
('og', 201, 'base', 19902, 'tot', 874)
(0.06351748567983118, 0.11693548387096775, 0.010099487488694603)
0.0635174856798
newRun5_44_1_-7_5_6_12
[5, 6, 12]
('og', 245, 'base', 19902, 'tot', 897)
(0.062102740930559765, 0.11189516129032262, 0.012310320570796904)
0.0621027409306
('delta', -0.0014068937795196426, 'starting', [5, 6, 12])
newRun5_44_1_-7_7_4_12
[7, 4, 12]
('og', 265, 'base', 19902, 'tot', 897)
(0.061597138478544866, 0.1098790322580645, 0.013315244699025224)
0.0615971384785
newRun5_44_1_-7_8_4_12
[8, 4, 12]
('og', 223, 'base', 19902, 'tot', 897)
(0.060541968143905125, 0.1098790322580645, 0.

95.832201004
newRun5_44_1_-6_4_6_12
[4, 6, 12]
('og', 301, 'base', 19902, 'tot', 888)
(0.06451769922620841, 0.11391129032258063, 0.015124108129836198)
0.0645176992262
newRun5_44_1_-6_3_6_12
[3, 6, 12]
('og', 422, 'base', 19902, 'tot', 888)
(0.06755759471409908, 0.11391129032258063, 0.021203899105617527)
0.0675575947141
('delta', 0.003039895487890673, 'starting', [3, 6, 12])
newRun5_44_1_-6_1_5_7
[1, 5, 7]
('og', 550, 'base', 19902, 'tot', 857)
(0.07782980353733294, 0.12802419354838712, 0.027635413526278765)
0.0778298035373
newRun5_44_1_-6_5_6_12
[5, 6, 12]
('og', 245, 'base', 19902, 'tot', 888)
(0.06311080544668876, 0.11391129032258063, 0.012310320570796904)
0.0631108054467
('delta', -0.0014068937795196496, 'starting', [5, 6, 12])
newRun5_44_1_-6_5_6_1
[5, 6, 1]
('og', 477, 'base', 19902, 'tot', 902)
(0.06490710732589688, 0.10584677419354838, 0.023967440458245403)
0.0649071073259
newRun5_44_1_-6_6_6_12
[6, 6, 12]
('og', 190, 'base', 19902, 'tot', 890)
(0.060720970254245825, 0.111895161

97.232175827
newRun5_44_1_-5_4_6_12
[4, 6, 12]
('og', 301, 'base', 19902, 'tot', 872)
(0.07157415083911166, 0.12802419354838712, 0.015124108129836198)
0.0715741508391
newRun5_44_1_-5_3_6_12
[3, 6, 12]
('og', 422, 'base', 19902, 'tot', 872)
(0.07461404632700233, 0.12802419354838712, 0.021203899105617527)
0.074614046327
('delta', 0.003039895487890673, 'starting', [3, 6, 12])
newRun5_44_1_-5_2_1_15
[2, 1, 15]
('og', 5560, 'base', 19902, 'tot', 899)
(0.19411993769470406, 0.1088709677419355, 0.27936890764747263)
0.194119937695
newRun5_44_1_-5_5_6_12
[5, 6, 12]
('og', 245, 'base', 19902, 'tot', 872)
(0.07016725705959201, 0.12802419354838712, 0.012310320570796904)
0.0701672570596
('delta', -0.0014068937795196496, 'starting', [5, 6, 12])
newRun5_44_1_-5_4_1_12
[4, 1, 12]
('og', 896, 'base', 19902, 'tot', 872)
(0.0865223972465079, 0.12802419354838712, 0.04502060094462868)
0.0865223972465
newRun5_44_1_-5_6_6_12
[6, 6, 12]
('og', 190, 'base', 19902, 'tot', 872)
(0.06878548638327807, 0.12802419354

('og', 455, 'base', 19902, 'tot', 866)
(0.07191488292633906, 0.12096774193548387, 0.022862023917194253)
0.0719148829263
newRun5_44_1_-5_9_6_15
[9, 6, 15]
('og', 216, 'base', 19902, 'tot', 893)
(0.06137417093759423, 0.11189516129032262, 0.010853180584865842)
0.0613741709376
('delta', 0.001207479147824382, 'starting', [9, 6, 15])
newRun5_44_1_-5_2_3_19
[2, 3, 19]
('og', 899, 'base', 19902, 'tot', 864)
(0.0835735730077379, 0.12197580645161288, 0.045171339563862926)
0.0835735730077
[(5.0, 7.0, 'mexhat_4.0_9x9.conv', 'ccd_44', 'median_1', -5, 1.0, 62.0, 57, 49, 49, 62.0, 62.0, 62.0, 50, 62.0, 48, 59, 30, 62.0, 58, 46, 47, 872, 1285, 301, 301, 0.07157415083911166), (4.0, 7.0, 'mexhat_4.0_9x9.conv', 'ccd_44', 'median_1', -5, 1.0, 62.0, 57, 49, 49, 62.0, 62.0, 62.0, 50, 62.0, 48, 59, 30, 62.0, 58, 46, 47, 872, 1401, 422, 422, 0.07461404632700233), (3.0, 2.0, 'tophat_2.0_3x3.conv', 'ccd_44', 'median_1', -5, 1.0, 62.0, 62.0, 62.0, 50, 62.0, 62.0, 62.0, 49, 62.0, 52, 59, 25, 62.0, 60, 47, 46, 899

151.376309156
newRun5_44_1_-4_4_6_12
[4, 6, 12]
('og', 301, 'base', 19902, 'tot', 890)
(0.06451769922620841, 0.11391129032258063, 0.015124108129836198)
0.0645176992262
newRun5_44_1_-4_3_6_12
[3, 6, 12]
('og', 422, 'base', 19902, 'tot', 891)
(0.06705356245603458, 0.11290322580645162, 0.021203899105617527)
0.067053562456
('delta', 0.0025358632298261713, 'starting', [3, 6, 12])
newRun5_44_1_-4_1_5_17
[1, 5, 17]
('og', 2912, 'base', 19902, 'tot', 896)
(0.12759396040598936, 0.1088709677419355, 0.14631695307004322)
0.127593960406
newRun5_44_1_-4_5_6_12
[5, 6, 12]
('og', 245, 'base', 19902, 'tot', 890)
(0.06311080544668876, 0.11391129032258063, 0.012310320570796904)
0.0631108054467
('delta', -0.0014068937795196496, 'starting', [5, 6, 12])
newRun5_44_1_-4_4_4_3
[4, 4, 3]
('og', 643, 'base', 19902, 'tot', 877)
(0.07260576826449604, 0.11290322580645162, 0.03230831072254045)
0.0726057682645
newRun5_44_1_-4_6_6_12
[6, 6, 12]
('og', 190, 'base', 19902, 'tot', 890)
(0.06172903477037483, 0.1139112903

125.877261162
newRun5_44_1_-3_4_6_12
[4, 6, 12]
('og', 301, 'base', 19902, 'tot', 892)
(0.06350963471007941, 0.11189516129032262, 0.015124108129836198)
0.0635096347101
newRun5_44_1_-3_3_6_12
[3, 6, 12]
('og', 422, 'base', 19902, 'tot', 892)
(0.06654953019797008, 0.11189516129032262, 0.021203899105617527)
0.066549530198
('delta', 0.003039895487890673, 'starting', [3, 6, 12])
newRun5_44_1_-3_4_5_10
[4, 5, 10]
('og', 285, 'base', 19902, 'tot', 855)
(0.08024476183298158, 0.14616935483870963, 0.014320168827253543)
0.080244761833
newRun5_44_1_-3_5_6_12
[5, 6, 12]
('og', 245, 'base', 19902, 'tot', 892)
(0.062102740930559765, 0.11189516129032262, 0.012310320570796904)
0.0621027409306
('delta', -0.0014068937795196426, 'starting', [5, 6, 12])
newRun5_44_1_-3_4_2_8
[4, 2, 8]
('og', 1166, 'base', 19902, 'tot', 822)
(0.11094676414430711, 0.16330645161290325, 0.05858707667571098)
0.110946764144
newRun5_44_1_-3_6_6_12
[6, 6, 12]
('og', 190, 'base', 19902, 'tot', 892)
(0.060720970254245825, 0.11189516

94.7966210842
newRun5_44_1_-2_4_6_12
[4, 6, 12]
('og', 301, 'base', 19902, 'tot', 889)
(0.06552576374233747, 0.11592741935483875, 0.015124108129836198)
0.0655257637423
newRun5_44_1_-2_3_6_12
[3, 6, 12]
('og', 422, 'base', 19902, 'tot', 889)
(0.06806162697216359, 0.11491935483870963, 0.021203899105617527)
0.0680616269722
('delta', 0.002535863229826116, 'starting', [3, 6, 12])
newRun5_44_1_-2_6_2_16
[6, 2, 16]
('og', 1070, 'base', 19902, 'tot', 898)
(0.07980510752688172, 0.10584677419354838, 0.053763440860215055)
0.0798051075269
newRun5_44_1_-2_5_6_12
[5, 6, 12]
('og', 245, 'base', 19902, 'tot', 887)
(0.06411886996281782, 0.11592741935483875, 0.012310320570796904)
0.0641188699628
('delta', -0.0014068937795196496, 'starting', [5, 6, 12])
newRun5_44_1_-2_1_1_13
[1, 1, 13]
('og', 1643, 'base', 19902, 'tot', 866)
(0.1032732263089137, 0.123991935483871, 0.08255451713395638)
0.103273226309
newRun5_44_1_-2_6_6_12
[6, 6, 12]
('og', 190, 'base', 19902, 'tot', 886)
(0.06324113154456838, 0.11693548

InconsistentTableError: Number of header columns (5) inconsistent with data columns (4) at data line 686
Header values: ['NUMBER', 'MAG_BEST', 'X_IMAGE', 'Y_IMAGE', 'ELLIPTICITY']
Data values: ['687', '20.9392', '798.1652', '43.3848']